In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
pd.set_option('display.max_columns', None)
%matplotlib inline

Read dei dataSet da GitHub

In [ ]:
url_prix = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/prix-consommation_cleaned.csv'
url_1er_leg = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/legislatives-2022-1er-tour_cleaned.csv'
url_1er_pres = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/presidentielles-2022-1er-tour_cleaned.csv'
url_2nd_pres = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/presidentielles-2022-2nd-tour_cleaned.csv'
url_vax = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/covid-19-france-vaccinations-age-sexe-dep_cleaned.csv'
url_job = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/labouref-france-departement-quarter-jobseeker_cleaned.csv'
url_tcrd = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/TCRD_025.csv'
url_inflazione = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/inflazionePerDipartimento.csv'
url_pib = 'https://raw.githubusercontent.com/manuelecapece/Unibas-Progetto-Big-data-2022-23/main/DataSet%20cleaned/PIBPerDipartimento.csv'
df_prix = pd.read_csv(url_prix, sep = ';') #ds andamento prezzi al consumo
df_1er_pres = pd.read_csv(url_1er_pres, sep = ';') #ds primo turno presidenziali
df_2nd_pres = pd.read_csv(url_2nd_pres, sep = ';') #ds secondo turno presidenziali
df_1er_leg = pd.read_csv(url_1er_leg, sep = ';') #ds primo turno legislative
df_vax = pd.read_csv(url_vax, sep = ';') #ds vaccinazioni covid
df_job = pd.read_csv(url_job, sep = ';') #ds disoccupazione
df_tcrd = pd.read_csv(url_tcrd, sep = ',') #ds disoccupazione per dipartimento
df_inflazione = pd.read_csv(url_inflazione, sep = ',') #ds inflazione per dipartimento
df_pib = pd.read_csv(url_pib, sep = ',') #ds reddito per dipartimento

<ipython-input-2-dc30c5596a9a>:11: DtypeWarning: Columns (0,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1er_pres = pd.read_csv(url_1er_pres, sep = ';') #ds primo turno presidenziali
<ipython-input-2-dc30c5596a9a>:13: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1er_leg = pd.read_csv(url_1er_leg, sep = ';') #ds primo turno legislative


In [ ]:
df_1er_pres.head(1)

In [ ]:
df_1er_leg.head(1)

In [ ]:
df_prix.head(1)

In [ ]:
df_vax.head(1)

In [ ]:
df_job.head(1)

In [ ]:
df_tcrd.head(1)

In [ ]:
df_inflazione.head(1)

In [ ]:
df_pib.head(1)

FILTRAGGIO SU ABITANTI FRANCESI CHE VIVONO ALL'ESTERO PER CSV PRESIDENZIALI PRIMO E SECONDO TURNO

In [ ]:
#ulteriore filtraggio per 'Français établis hors de France'
condizione = df_1er_pres['Libellé du département'] == 'Français établis hors de France'
df_1er_pres.drop(df_1er_pres[condizione].index, inplace = True)
condizione2 = df_2nd_pres['Libellé du département'] == 'Français établis hors de France'
df_2nd_pres.drop(df_2nd_pres[condizione2].index, inplace = True)

# Query prefissate

### Cambio tipi di dato

In [ ]:
df_job[['Trimestre','Catégorie','Tranche d\'âge','Nom Officiel Région','Code Iso 3166-3 Zone','Nom Officiel Département','Code Officiel Département']] = df_job[['Trimestre','Catégorie','Tranche d\'âge','Nom Officiel Région','Code Iso 3166-3 Zone','Nom Officiel Département','Code Officiel Département']].astype("string")
df_job['Période'] = df_job['Période'].astype('int64')
df_job['Date'] = pd.to_datetime(df_job['Date'])
df_job.rename(columns={'Tranche d\'âge': 'Tranche_dage','Nom Officiel Département':'Libellé du département', 'Code Officiel Département':'Code du département'}, inplace = True)

In [ ]:
df_vax[['Nom Officiel Région','Code Iso 3166-3 Zone','Nom Officiel Département','Code Officiel Département']] = df_vax[['Nom Officiel Région','Code Iso 3166-3 Zone','Nom Officiel Département','Code Officiel Département']].astype("string")
df_vax['Date'] = pd.to_datetime(df_vax['Date'])
df_vax.rename(columns={'Nom Officiel Département':'Libellé du département'}, inplace = True)

In [ ]:
df_tcrd[['Libellé du département','Code du département']] = df_tcrd[['Libellé du département','Code du département']].astype("string")

In [ ]:
df_inflazione[["Code du département", "Libellé du département",'Nom Officiel Région']] = df_inflazione[[
    "Code du département", "Libellé du département",'Nom Officiel Région']].astype("string")

In [ ]:
def cambiaTipo(df):
  df[['Code du département','Code de la commune', 'Code du b.vote']] = df[['Code du département','Code de la commune', 'Code du b.vote']].astype("string")
  df[['Libellé du département','Libellé de la circonscription', 'Libellé de la commune',
    'Sexe','Nom','Prénom','Nom Officiel Région','scrutin_code']] = df[['Libellé du département','Libellé de la circonscription', 'Libellé de la commune',
                                                        'Sexe','Nom','Prénom','Nom Officiel Région','scrutin_code']].astype("string")
  return df

In [ ]:
df_1er_pres = cambiaTipo(df_1er_pres)
df_1er_leg = cambiaTipo(df_1er_leg)
df_2er_pres = cambiaTipo(df_2nd_pres)

## 1) Query sulle percentuali di votazioni

In [ ]:
#funzione percentuale voti candidato per dipartimento
def getVoixPercDipa(nome, dataset):
    df_filtered = dataset[dataset['Nom'] == nome]
    df_leg = df_filtered[['Nom', 'Libellé du département','Exprimés','Voix','% Voix/Exp']]
    percentuale_per_dipartimento = df_leg.groupby('Libellé du département')[['Exprimés','Voix']].sum().sort_values(['Libellé du département'])
    df = (percentuale_per_dipartimento['Voix'] / percentuale_per_dipartimento['Exprimés'])*100
    percentuale_per_dipartimento['% Voix/Exp'] = df
    percentuale_per_dipartimento = percentuale_per_dipartimento.rename(columns={'% Voix/Exp': '% Voix/Exp ' + nome})
    df_merged = pd.merge(percentuale_per_dipartimento, dataset, on='Libellé du département')
    return df_merged

### 1.1) I 20 dipartimenti con la maggiore percentuale di voti per Macron

#### Primo turno

In [ ]:
#primo turno
df = getVoixPercDipa('MACRON', df_1er_pres)
df_MACRON_MAGGIORE_PRIMO = df[(df.Nom == 'MACRON')][['Nom', 'Code du département', 'Libellé du département', 'Nom Officiel Région', '% Voix/Exp MACRON']].sort_values(by = ['% Voix/Exp MACRON'], ascending = False)
df_MACRON_MAGGIORE_PRIMO.drop_duplicates(subset='Libellé du département', inplace=True)
df_MACRON_MAGGIORE_PRIMO.head(20)

#### Secondo turno

In [ ]:
#secondo turno
df2 = getVoixPercDipa('MACRON', df_2nd_pres)
df_MACRON_MAGGIORE_SECONDO = df2[(df2.Nom == 'MACRON')][['Nom', 'Code du département', 'Libellé du département', 'Nom Officiel Région', '% Voix/Exp MACRON']].sort_values(by = ['% Voix/Exp MACRON'], ascending = False)
df_MACRON_MAGGIORE_SECONDO.drop_duplicates(subset='Libellé du département', inplace=True)
df_MACRON_MAGGIORE_SECONDO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp MACRON
49083,MACRON,97,Nouvelle-Calédonie,Saint-Martin,86.666667
42228,MACRON,44,Loire-Atlantique,Pays de la Loire,69.477371
54447,MACRON,69,Rhône,Auvergne-Rhône-Alpes,68.647062
46071,MACRON,49,Maine-et-Loire,Pays de la Loire,66.528758
26775,MACRON,31,Haute-Garonne,Occitanie,64.418940
48709,MACRON,53,Mayenne,Pays de la Loire,64.207724
52246,MACRON,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine,63.054115
16673,MACRON,79,Deux-Sèvres,Nouvelle-Aquitaine,62.131273
63636,MACRON,85,Vendée,Pays de la Loire,61.855561
31117,MACRON,74,Haute-Savoie,Auvergne-Rhône-Alpes,61.690065


### 1.1) I 20 dipartimenti con la minore: percentuale di voti per Macron

#### Primo turno

In [ ]:
# primo turno
df_2 = getVoixPercDipa('MACRON', df_1er_pres)
df_MACRON_MINORE_PRIMO = df_2[(df_2.Nom == 'MACRON')][['Nom', 'Code du département', 'Libellé du département', 'Nom Officiel Région', '% Voix/Exp MACRON']].sort_values(by = ['% Voix/Exp MACRON'], ascending = True)
df_MACRON_MINORE_PRIMO.drop_duplicates(subset='Libellé du département', inplace=True)
df_MACRON_MINORE_PRIMO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp MACRON
149004,MACRON,97,Guadeloupe,Guadeloupe,13.426361
153851,MACRON,97,Guyane,Guyane,14.224366
286662,MACRON,97,Martinique,Martinique,16.301881
292615,MACRON,97,Mayotte,Mayotte,16.935806
158453,MACRON,2B,Haute-Corse,Corse,17.578665
224915,MACRON,97,La Réunion,La Réunion,18.035794
90032,MACRON,2A,Corse-du-Sud,Corse,18.750179
294502,MACRON,97,Nouvelle-Calédonie,Saint-Martin,19.230769
34873,MACRON,9,Ariège,Occitanie,19.705793
335871,MACRON,97,Saint-Pierre-et-Miquelon,Saint-Pierre-et-Miquelon,19.770455


#### Secondo turno

In [ ]:
#secondo turno
df2_2 = getVoixPercDipa('MACRON', df_2nd_pres)
df2_MACRON_MINORE_SECONDO = df2_2[(df2_2.Nom == 'MACRON')][['Nom', 'Code du département', 'Libellé du département','Nom Officiel Région', '% Voix/Exp MACRON']].sort_values(by = ['% Voix/Exp MACRON'], ascending = True)
df2_MACRON_MINORE_SECONDO.drop_duplicates(subset='Libellé du département', inplace=True)
df2_MACRON_MINORE_SECONDO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp MACRON
25384,MACRON,97,Guadeloupe,Guadeloupe,30.399365
47648,MACRON,97,Martinique,Martinique,39.125069
25773,MACRON,97,Guyane,Guyane,39.222024
39283,MACRON,97,La Réunion,La Réunion,40.436881
49027,MACRON,97,Mayotte,Mayotte,40.904120
15339,MACRON,2A,Corse-du-Sud,Corse,41.691947
26429,MACRON,2B,Haute-Corse,Corse,42.173304
52510,MACRON,66,Pyrénées-Orientales,Occitanie,43.666676
55975,MACRON,97,Saint-Martin/Saint-Barthélemy,Saint-Martin,44.575663
60959,MACRON,83,Var,Provence-Alpes-Côte d'Azur,44.899504


### 1.2) I 20 dipartimenti con la maggiore percentuale di voti per Le Pen



#### Primo turno

In [ ]:
#primo turno
dfLP = getVoixPercDipa('LE PEN', df_1er_pres)
dfLP_LEPEN_MAGGIORE_PRIMO = dfLP[(dfLP.Nom == 'LE PEN')][['Nom', 'Code du département', 'Libellé du département', 'Nom Officiel Région', '% Voix/Exp LE PEN']].sort_values(by = ['% Voix/Exp LE PEN'], ascending = False)
dfLP_LEPEN_MAGGIORE_PRIMO.drop_duplicates(subset='Libellé du département', inplace=True)
dfLP_LEPEN_MAGGIORE_PRIMO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp LE PEN
293624,LE PEN,97,Mayotte,Mayotte,42.676177
317506,LE PEN,66,Pyrénées-Orientales,Occitanie,32.740312
364140,LE PEN,83,Var,Provence-Alpes-Côte d'Azur,30.612630
38351,LE PEN,11,Aude,Occitanie,30.140849
371443,LE PEN,84,Vaucluse,Provence-Alpes-Côte d'Azur,29.425238
119468,LE PEN,30,Gard,Occitanie,29.344484
91214,LE PEN,2A,Corse-du-Sud,Corse,29.184427
357944,LE PEN,82,Tarn-et-Garonne,Occitanie,28.928131
155227,LE PEN,2B,Haute-Corse,Corse,27.967655
342227,LE PEN,72,Sarthe,Pays de la Loire,27.682437


#### Secondo turno

In [ ]:
#secondo turno
dfLP2 = getVoixPercDipa('LE PEN', df_2nd_pres)
dfLP_LEPEN_MAGGIORE_SECONDO = dfLP2[(dfLP2.Nom == 'LE PEN')][['Nom', 'Code du département', 'Libellé du département', 'Nom Officiel Région', '% Voix/Exp LE PEN']].sort_values(by = ['% Voix/Exp LE PEN'], ascending = False)
dfLP_LEPEN_MAGGIORE_SECONDO.drop_duplicates(subset='Libellé du département', inplace=True)
dfLP_LEPEN_MAGGIORE_SECONDO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp LE PEN
25380,LE PEN,97,Guadeloupe,Guadeloupe,69.600635
47657,LE PEN,97,Martinique,Martinique,60.874931
25808,LE PEN,97,Guyane,Guyane,60.777976
39280,LE PEN,97,La Réunion,La Réunion,59.563119
49040,LE PEN,97,Mayotte,Mayotte,59.095880
15323,LE PEN,2A,Corse-du-Sud,Corse,58.308053
26452,LE PEN,2B,Haute-Corse,Corse,57.826696
52535,LE PEN,66,Pyrénées-Orientales,Occitanie,56.333324
55973,LE PEN,97,Saint-Martin/Saint-Barthélemy,Saint-Martin,55.424337
60955,LE PEN,83,Var,Provence-Alpes-Côte d'Azur,55.100496


### 1.2) I 20 dipartimenti con la minore percentuale di voti per Le Pen


#### Primo turno

In [ ]:
dfLP = getVoixPercDipa('LE PEN', df_1er_pres)
dfLP_LEPEN_MINORE_PRIMO = dfLP[(dfLP.Nom == 'LE PEN')][['Nom', 'Code du département', 'Libellé du département','Nom Officiel Région', '% Voix/Exp LE PEN']].sort_values(by = ['% Voix/Exp LE PEN'], ascending = True)
dfLP_LEPEN_MINORE_PRIMO.drop_duplicates(subset='Libellé du département', inplace=True)
dfLP_LEPEN_MINORE_PRIMO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp LE PEN
294495,LE PEN,97,Nouvelle-Calédonie,Saint-Martin,5.769231
284684,LE PEN,97,Martinique,Martinique,13.416132
328757,LE PEN,69,Rhône,Auvergne-Rhône-Alpes,16.552349
252359,LE PEN,44,Loire-Atlantique,Pays de la Loire,16.911550
335901,LE PEN,97,Saint-Pierre-et-Miquelon,Saint-Pierre-et-Miquelon,16.993706
335554,LE PEN,97,Saint-Martin/Saint-Barthélemy,Saint-Martin,17.268164
307654,LE PEN,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine,17.377270
154141,LE PEN,97,Guyane,Guyane,17.662642
151086,LE PEN,97,Guadeloupe,Guadeloupe,17.917607
168296,LE PEN,31,Haute-Garonne,Occitanie,18.229525


#### Secondo turno

In [ ]:
dfLP2 = getVoixPercDipa('LE PEN', df_2nd_pres)
dfLP_LEPEN_MINORE_SECONDO = dfLP2[(dfLP2.Nom == 'LE PEN')][['Nom', 'Code du département', 'Libellé du département', 'Nom Officiel Région', '% Voix/Exp LE PEN']].sort_values(by = ['% Voix/Exp LE PEN'], ascending = True)
dfLP_LEPEN_MINORE_SECONDO.drop_duplicates(subset='Libellé du département', inplace=True)
dfLP_LEPEN_MINORE_SECONDO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp LE PEN
49082,LE PEN,97,Nouvelle-Calédonie,Saint-Martin,13.333333
42158,LE PEN,44,Loire-Atlantique,Pays de la Loire,30.522629
54493,LE PEN,69,Rhône,Auvergne-Rhône-Alpes,31.352938
46079,LE PEN,49,Maine-et-Loire,Pays de la Loire,33.471242
26829,LE PEN,31,Haute-Garonne,Occitanie,35.581060
48697,LE PEN,53,Mayenne,Pays de la Loire,35.792276
52251,LE PEN,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine,36.945885
16622,LE PEN,79,Deux-Sèvres,Nouvelle-Aquitaine,37.868727
63565,LE PEN,85,Vendée,Pays de la Loire,38.144439
31101,LE PEN,74,Haute-Savoie,Auvergne-Rhône-Alpes,38.309935


### 1.3) I 20 dipartimenti con la maggiore percentuale di voti per Jean-Luc Melenchon

#### Primo turno

In [ ]:
dfJLM = getVoixPercDipa('MÉLENCHON', df_1er_pres)
df_MELENCHON_MAGGIORE_PRIMO = dfJLM[(dfJLM.Nom == 'MÉLENCHON')][['Nom', 'Code du département', 'Libellé du département', 'Nom Officiel Région', '% Voix/Exp MÉLENCHON']].sort_values(by = ['% Voix/Exp MÉLENCHON'], ascending = False)
df_MELENCHON_MAGGIORE_PRIMO.drop_duplicates(subset='Libellé du département', inplace=True)
df_MELENCHON_MAGGIORE_PRIMO.head(20)

,Nom,Code du département,Libellé du département,Nom Officiel Région,% Voix/Exp MÉLENCHON
294501,MÉLENCHON,97,Nouvelle-Calédonie,Saint-Martin,57.692308
151466,MÉLENCHON,97,Guadeloupe,Guadeloupe,56.158715
286565,MÉLENCHON,97,Martinique,Martinique,53.104946
154715,MÉLENCHON,97,Guyane,Guyane,50.592566
335861,MÉLENCHON,97,Saint-Pierre-et-Miquelon,Saint-Pierre-et-Miquelon,40.910774
229620,MÉLENCHON,97,La Réunion,La Réunion,40.258849
335828,MÉLENCHON,97,Saint-Martin/Saint-Barthélemy,Saint-Martin,28.130975
33817,MÉLENCHON,9,Ariège,Occitanie,26.070837
173421,MÉLENCHON,31,Haute-Garonne,Occitanie,25.837541
327696,MÉLENCHON,69,Rhône,Auvergne-Rhône-Alpes,25.196119


## 2) Query sulla disoccupazione

### 2.1)  I 20 dipartimenti con il maggior tasso di disoccupazione




In [ ]:
df_tcrd = df_tcrd.sort_values('% Taux de chômage département T4',ascending=False)
df_tcrd.head(20)

### 2.2)  I 20 dipartimenti con il minor tasso di disoccupazione

In [ ]:
df_tcrd = df_tcrd.sort_values('% Taux de chômage département T4',ascending=True)
df_tcrd.head(20)

## 3) Query sul reddito dichiarato pro capite

### 3.1)  I 20 dipartimenti con il maggior reddito pro *capite*



In [ ]:
df_pib = df_pib.sort_values('PIB en valeur sont à prix courants',ascending=False)
df_pib.head(20)

,Code du département,Libellé du département,Nom Officiel Région,PIB en valeur sont à prix courants
24,74,Haute-Savoie,Auvergne-Rhône-Alpes,36766
0,1,Ain,Auvergne-Rhône-Alpes,31232
2,6,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,28938
32,44,Loire-Atlantique,Pays de la Loire,28841
43,73,Savoie,Auvergne-Rhône-Alpes,28807
22,31,Haute-Garonne,Occitanie,28714
29,38,Isère,Auvergne-Rhône-Alpes,28666
20,33,Gironde,Nouvelle-Aquitaine,27909
39,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine,26918
46,83,Var,Provence-Alpes-Côte d'Azur,26870


### 3.2)  I 20 dipartimenti con il minor reddito pro *capite*

In [ ]:
df_pib = df_pib.sort_values('PIB en valeur sont à prix courants',ascending=True)
df_pib.head(20)

,Code du département,Libellé du département,Nom Officiel Région,PIB en valeur sont à prix courants
14,23,Creuse,Nouvelle-Aquitaine,20301
5,9,Ariège,Occitanie,21250
6,11,Aude,Occitanie,21669
35,48,Lozère,Occitanie,22179
40,66,Pyrénées-Orientales,Occitanie,22342
9,15,Cantal,Auvergne-Rhône-Alpes,22420
27,65,Hautes-Pyrénées,Occitanie,22453
1,3,Allier,Auvergne-Rhône-Alpes,22556
16,24,Dordogne,Nouvelle-Aquitaine,22637
34,47,Lot-et-Garonne,Nouvelle-Aquitaine,22774


## 4) Query sulla percentuale di vaccinazioni COVID


### 4.1)  I 20 dipartimenti con la maggiore percentuale di vaccinazioni COVID


In [ ]:
df_vax = (df_vax[['Libellé du département', 'Nom Officiel Région', 'Couverture vaccinale complète (dose n°2)']]).sort_values('Libellé du département').sort_values(by = ['Couverture vaccinale complète (dose n°2)'], ascending = False)
df_vax.head(20)

### 4.2)  I 20 dipartimenti con la minor percentuale di vaccinazioni COVID


In [ ]:
df_vax = (df_vax[['Libellé du département', 'Nom Officiel Région', 'Couverture vaccinale complète (dose n°2)']]).sort_values('Libellé du département').sort_values(by = ['Couverture vaccinale complète (dose n°2)'], ascending = True)
df_vax.head(20)

## 5) Query sull'inflazione

### 5.1) 10 dipartimenti della Francia Metropolitana con la maggiore inflazione

In [ ]:
df = df_inflazione.sort_values('Indice des prix dans le département',ascending=False)
df.head(10)

### 5.2) I 10 dipartimenti della Francia Metropolitana con la minore inflazione

In [ ]:
df = df_inflazione.sort_values('Indice des prix dans le département',ascending=True)
df.head(10)


## 6) Query sulle differenze di votazione

#### Definizione delle funzioni per trovare i dipartimenti con i maggiori cambiamenti di voti tra presidenziali e legislative

In [ ]:
def trovaSommaVotiPerDipartimentoPres(nome, dataFrame):
  df_filtered = dataFrame[dataFrame['Nom'] == nome]
  df_pres = df_filtered[['Nom', 'Libellé du département','Voix']]
  somma_per_dipartimento = df_pres.groupby('Libellé du département')[['Voix']].sum().sort_values(['Voix'] , ascending=False)
  somma_per_dipartimento = somma_per_dipartimento.rename(columns={'Voix': 'Voix ' + nome})
  return somma_per_dipartimento

def trovaSommaVotiPerDipartimentoLeg(nome, dataFrame):
  df_filtered = dataFrame[dataFrame['Nuance'] == nome]
  df_leg = df_filtered[['Nom','Nuance', 'Libellé du département','Voix']]
  somma_per_dipartimento = df_leg.groupby('Libellé du département')[['Voix']].sum().sort_values(['Voix'] , ascending=False)
  somma_per_dipartimento = somma_per_dipartimento.rename(columns={'Voix': 'Voix ' + nome})
  return somma_per_dipartimento

def creaDfDifferenza(nomePresidenziali, nomeLegislative, dfPresidenziali, dfLegislative):
  somma_per_dipartimento_pres = trovaSommaVotiPerDipartimentoPres(nomePresidenziali,dfPresidenziali)
  somma_per_dipartimento_leg = trovaSommaVotiPerDipartimentoLeg(nomeLegislative,dfLegislative)
  df_differenza = pd.concat([somma_per_dipartimento_pres, somma_per_dipartimento_leg], axis=1).fillna(0)
  temp = df_differenza['Voix '+ nomePresidenziali] - df_differenza['Voix '+ nomeLegislative]
  df_differenza['Abs differenza'] = temp.abs()
  return df_differenza

### 6.1) I 10 dipartimenti con il maggiore cambiamento di voti tra il primo turno delle presidenziali e il primo turno delle legislative per Macron/Ensemble

In [ ]:
dfDifferenza_Macron_ENS =creaDfDifferenza('MACRON','ENS', df_1er_pres, df_1er_leg)
dfDifferenza_Macron_ENS.head(10)

,Voix MACRON,Voix ENS,Abs differenza
Libellé du département,,,
Rhône,278243.0,176400.0,101843.0
Loire-Atlantique,256609.0,153309.0,103300.0
Gironde,256287.0,176705.0,79582.0
Bouches-du-Rhône,229038.0,136364.0,92674.0
Haute-Garonne,196712.0,133853.0,62859.0
Isère,180533.0,119348.0,61185.0
Maine-et-Loire,157063.0,96896.0,60167.0
Vendée,149587.0,80712.0,68875.0
Var,142335.0,101482.0,40853.0


### 6.2) I 10 dipartimenti con il maggiore cambiamento di voti tra il primo turno delle presidenziali e il primo turno delle legislative per Le Pen/RN

In [ ]:
dfDifferenza_LePen_RN =creaDfDifferenza('LE PEN','RN', df_1er_pres, df_1er_leg)
dfDifferenza_LePen_RN.head(10)

,Voix LE PEN,Voix RN,Abs differenza
Libellé du département,,,
Bouches-du-Rhône,264753.0,152252.0,112501.0
Gironde,191694.0,114222.0,77472.0
Var,183287.0,111820.0,71467.0
Hérault,165734.0,83235.0,82499.0
Isère,154799.0,84317.0,70482.0
Rhône,150463.0,71414.0,79049.0
Alpes-Maritimes,148919.0,70829.0,78090.0
Loire-Atlantique,135702.0,65508.0,70194.0
Haute-Garonne,133254.0,78380.0,54874.0


### 6.3) I 10 dipartimenti con il maggiore cambiamento di voti tra il primo turno delle presidenziali e il primo turno delle legislative per Melenchon/Nupes

In [ ]:
dfDifferenza_Melechon_NUP =creaDfDifferenza('MÉLENCHON','NUP', df_1er_pres, df_1er_leg)
dfDifferenza_Melechon_NUP.head(10)

,Voix MÉLENCHON,Voix NUP,Abs differenza
Libellé du département,,,
Bouches-du-Rhône,237971.0,161271.0,76700.0
Rhône,229036.0,168857.0,60179.0
Gironde,194868.0,173518.0,21350.0
Haute-Garonne,188867.0,153424.0,35443.0
Loire-Atlantique,187977.0,182620.0,5357.0
Hérault,154819.0,105721.0,49098.0
Isère,153474.0,132151.0,21323.0
La Réunion,139604.0,0.0,139604.0
Alpes-Maritimes,92345.0,58127.0,34218.0


### 6.4) I 10 dipartimenti con il maggiore cambiamento di voti tra il primo turno delle presidenziali e il primo turno delle legislative per Jadot/Nupes

In [ ]:
dfDifferenza_Jadot_NUP =creaDfDifferenza('JADOT','NUP', df_1er_pres, df_1er_leg)
dfDifferenza_Jadot_NUP.head(10)

,Voix JADOT,Voix NUP,Abs differenza
Libellé du département,,,
Loire-Atlantique,60072.0,182620.0,122548.0
Rhône,51907.0,168857.0,116950.0
Gironde,46690.0,173518.0,126828.0
Haute-Garonne,42599.0,153424.0,110825.0
Bouches-du-Rhône,42109.0,161271.0,119162.0
Isère,40378.0,132151.0,91773.0
Haute-Savoie,29948.0,57388.0,27440.0
Hérault,28057.0,105721.0,77664.0
Maine-et-Loire,26635.0,75891.0,49256.0


### 6.5) I 10 dipartimenti con il maggiore cambiamento di voti tra il primo turno delle presidenziali e il primo turno delle legislative per Hidalgo/Nupes

In [ ]:
dfDifferenza_Hidalgo_NUP =creaDfDifferenza('HIDALGO','NUP', df_1er_pres, df_1er_leg)
dfDifferenza_Hidalgo_NUP.head(10)

,Voix HIDALGO,Voix NUP,Abs differenza
Libellé du département,,,
Haute-Garonne,19658.0,153424.0,133766.0
Gironde,19607.0,173518.0,153911.0
Loire-Atlantique,18369.0,182620.0,164251.0
Rhône,15895.0,168857.0,152962.0
Isère,12736.0,132151.0,119415.0
Hérault,12150.0,105721.0,93571.0
Bouches-du-Rhône,11760.0,161271.0,149511.0
Pyrénées-Atlantiques,9276.0,64589.0,55313.0
Landes,9071.0,53854.0,44783.0
